In [50]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import sys
import csv
import functools
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

csv.field_size_limit(1000000000000000000)

1000000000000000000

In [51]:
df = pd.read_pickle("./3_MASTER_DEVICE_File_wSTDDEVICE_2000to2013.pkl")

In [52]:
for col in df.columns:
    
    if "DATE" in col:
        print(col)
# decide to use DATE_RECEIVED

DATE_OF_EVENT
DATE_ADDED
DATE_CHANGED
DATE_RECEIVED


In [53]:
df["DATE_RECEIVED"].value_counts()

2008/05/08    1183
2009/03/10    1124
2008/01/10     908
2007/11/09     808
2012/03/10     791
              ... 
2010/02/25       1
2010/02/18       1
2003/04/10       1
2002/07/11       1
2009/03/31       1
Name: DATE_RECEIVED, Length: 2016, dtype: int64

In [55]:
# converting to datetime
df["DATE_RECEIVED"].str.replace(pat="/", repl="-")

28790     2007-07-31
30694     2007-08-14
30721     2007-08-22
31540     2007-09-17
31541     2007-09-17
             ...    
169959    2013-09-10
173773    2013-10-16
175072    2013-11-04
175089    2013-11-04
175808    2013-11-07
Name: DATE_RECEIVED, Length: 76433, dtype: object

In [56]:
df["DATE_RECEIVED"] = pd.to_datetime(df["DATE_RECEIVED"])
df["DATE_RECEIVED"].dtype

dtype('<M8[ns]')

In [57]:
# creating the censored column
df["CENSORED"] = df["DATE_RECEIVED"]

In [58]:
# checking valuecounts
df[df["RECALL_CLASS"] != "0"]["STD_DEVICE"].value_counts()

MEDTRONIC_SPRINT_QUATTRO_SECURE    15098
MEDTRONIC_SPRINT_FIDELIS           10122
STJUDE_RIATA                        9539
STJUDE_RIATA_ST                     2999
SORIN_ISOLINE                        405
Name: STD_DEVICE, dtype: int64

In [80]:
# assigning censor tags
dfList = []
dfListNoCen = []

df_NOT_RECALLED = df[df["RECALL_CLASS"] == "0"]
df_NOT_RECALLED.assign(CENSOR="NO")

print("not recalled shape: " + str(df_NOT_RECALLED.shape))

not recalled shape: (38270, 34)


In [81]:
df_MEDTRONIC_SPRINT_QUATTRO_SECURE_censor = df[(df["DATE_RECEIVED"] >= datetime(2010, 11, 26)) & (df["STD_DEVICE"] == "MEDTRONIC_SPRINT_QUATTRO_SECURE")]
df_MEDTRONIC_SPRINT_QUATTRO_SECURE_censor.assign(CENSOR="YES")
dfList.append(df_MEDTRONIC_SPRINT_QUATTRO_SECURE_censor)

df_MEDTRONIC_SPRINT_QUATTRO_SECURE_nocensor = df[(df["DATE_RECEIVED"] < datetime(2010, 11, 26)) & (df["STD_DEVICE"] == "MEDTRONIC_SPRINT_QUATTRO_SECURE")]
df_MEDTRONIC_SPRINT_QUATTRO_SECURE_nocensor.assign(CENSOR="NO")
dfList.append(df_MEDTRONIC_SPRINT_QUATTRO_SECURE_nocensor)
dfListNoCen.append(df_MEDTRONIC_SPRINT_QUATTRO_SECURE_nocensor)

print("censored shape: " + str(df_MEDTRONIC_SPRINT_QUATTRO_SECURE_censor.shape))
print("non-censored shape: " + str(df_MEDTRONIC_SPRINT_QUATTRO_SECURE_nocensor.shape))

censored shape: (10959, 34)
non-censored shape: (4139, 34)


In [82]:
df_MEDTRONIC_SPRINT_FIDELIS_censor = df[(df["DATE_RECEIVED"] >= datetime(2007, 10, 25)) & (df["STD_DEVICE"] == "MEDTRONIC_SPRINT_FIDELIS")]
df_MEDTRONIC_SPRINT_FIDELIS_censor.assign(CENSOR="YES")
dfList.append(df_MEDTRONIC_SPRINT_FIDELIS_censor)

df_MEDTRONIC_SPRINT_FIDELIS_nocensor = df[(df["DATE_RECEIVED"] < datetime(2007, 10, 25)) & (df["STD_DEVICE"] == "MEDTRONIC_SPRINT_FIDELIS")]
df_MEDTRONIC_SPRINT_FIDELIS_nocensor.assign(CENSOR="NO")
dfList.append(df_MEDTRONIC_SPRINT_FIDELIS_nocensor)
dfListNoCen.append(df_MEDTRONIC_SPRINT_FIDELIS_nocensor)

print("censored shape: " + str(df_MEDTRONIC_SPRINT_FIDELIS_censor.shape))
print("non-censored shape: " + str(df_MEDTRONIC_SPRINT_FIDELIS_nocensor.shape))

censored shape: (8393, 34)
non-censored shape: (1729, 34)


In [83]:
df_STJUDE_RIATA_censor = df[(df["DATE_RECEIVED"] >= datetime(2012, 1, 11)) & (df["STD_DEVICE"] == "STJUDE_RIATA")]
df_STJUDE_RIATA_censor.assign(CENSOR="YES")
dfList.append(df_STJUDE_RIATA_censor)

df_STJUDE_RIATA_nocensor = df[(df["DATE_RECEIVED"] < datetime(2012, 1, 11)) & (df["STD_DEVICE"] == "STJUDE_RIATA")]
df_STJUDE_RIATA_nocensor.assign(CENSOR="NO")
dfList.append(df_STJUDE_RIATA_nocensor)
dfListNoCen.append(df_STJUDE_RIATA_nocensor)

print("censored shape: " + str(df_STJUDE_RIATA_censor.shape))
print("non-censored shape: " + str(df_STJUDE_RIATA_nocensor.shape))

censored shape: (5544, 34)
non-censored shape: (3995, 34)


In [84]:
df_STJUDE_RIATA_ST_censor = df[(df["DATE_RECEIVED"] >= datetime(2012, 1, 11)) & (df["STD_DEVICE"] == "STJUDE_RIATA_ST")]
df_STJUDE_RIATA_ST_censor.assign(CENSOR="YES")
dfList.append(df_STJUDE_RIATA_ST_censor)

df_STJUDE_RIATA_ST_nocensor = df[(df["DATE_RECEIVED"] < datetime(2012, 1, 11)) & (df["STD_DEVICE"] == "STJUDE_RIATA_ST")]
df_STJUDE_RIATA_ST_nocensor.assign(CENSOR="NO")
dfList.append(df_STJUDE_RIATA_ST_nocensor)
dfListNoCen.append(df_STJUDE_RIATA_ST_nocensor)

print("censored shape: " + str(df_STJUDE_RIATA_ST_censor.shape))
print("non-censored shape: " + str(df_STJUDE_RIATA_ST_nocensor.shape))

censored shape: (1507, 34)
non-censored shape: (1492, 34)


In [85]:
df_SORIN_ISOLINE_censor = df[(df["DATE_RECEIVED"] >= datetime(2013, 3, 8)) & (df["STD_DEVICE"] == "SORIN_ISOLINE")]
df_SORIN_ISOLINE_censor.assign(CENSOR="YES")
dfList.append(df_SORIN_ISOLINE_censor)

df_SORIN_ISOLINE_nocensor = df[(df["DATE_RECEIVED"] < datetime(2013, 3, 8)) & (df["STD_DEVICE"] == "SORIN_ISOLINE")]
df_SORIN_ISOLINE_nocensor.assign(CENSOR="NO")
dfList.append(df_SORIN_ISOLINE_nocensor)
dfListNoCen.append(df_SORIN_ISOLINE_nocensor)

print("censored shape: " + str(df_SORIN_ISOLINE_censor.shape))
print("non-censored shape: " + str(df_SORIN_ISOLINE_nocensor.shape))

censored shape: (67, 34)
non-censored shape: (338, 34)


In [87]:
# merging all the dfs together
for df in dfList:
    print(df.STD_DEVICE.iloc[0])

MEDTRONIC_SPRINT_QUATTRO_SECURE
MEDTRONIC_SPRINT_QUATTRO_SECURE
MEDTRONIC_SPRINT_FIDELIS
MEDTRONIC_SPRINT_FIDELIS
STJUDE_RIATA
STJUDE_RIATA
STJUDE_RIATA_ST
STJUDE_RIATA_ST
SORIN_ISOLINE
SORIN_ISOLINE


In [90]:
df_all = df_NOT_RECALLED
print(df_all.shape)

for df in dfList:
    df_all = df_all.append(df)
    print(df_all.shape)

(38270, 34)
(49229, 34)
(53368, 34)
(61761, 34)
(63490, 34)
(69034, 34)
(73029, 34)
(74536, 34)
(76028, 34)
(76095, 34)
(76433, 34)


In [94]:
print("the shape of df_all is: " + str(df_all.shape))
df_all.to_pickle("./3_MASTER_DEVICE_File_wCENSOR_all")

the shape of df_all is: (76433, 34)


In [88]:
# merging all the non-censored dfs together
for df in dfListNoCen:
    print(df.STD_DEVICE.iloc[0])

MEDTRONIC_SPRINT_QUATTRO_SECURE
MEDTRONIC_SPRINT_FIDELIS
STJUDE_RIATA
STJUDE_RIATA_ST
SORIN_ISOLINE


In [92]:
df_all_NoCen = df_NOT_RECALLED
print(df_all_NoCen.shape)

for df in dfListNoCen:
    df_all_NoCen = df_all_NoCen.append(df)
    print(df_all_NoCen.shape)

(38270, 34)
(42409, 34)
(44138, 34)
(48133, 34)
(49625, 34)
(49963, 34)


In [95]:
print("the shape of df_all_NoCen is: " + str(df_all_NoCen.shape))
df_all_NoCen.to_pickle("./3_MASTER_DEVICE_File_wCENSOR_NOTCENSORED")

the shape of df_all_NoCen is: (49963, 34)
